In [2]:
import pandas as pd
import numpy as np
import glob
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, f1_score, accuracy_score, precision_score, recall_score
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Define the path to the directory containing the feature files
directory_path = '/content/drive/MyDrive/FYP_dataset/features'

In [5]:
# Get a list of all feature file paths in the directory
feature_files = glob.glob(directory_path + '/*.csv')

In [6]:
# Define lists to store the feature and target data
X = []
y_a = []
y_v = []

In [7]:

# Iterate over each feature file
for file in feature_files:
    # Read the feature file into a DataFrame
    df = pd.read_csv(file)

    # Extract the features and target variables
    features = df.drop(['Arousal_Value', 'Valence_Value', 'frameTime'], axis=1).values
    arousal = df['Arousal_Value'].values
    valence = df['Valence_Value'].values

    # Append the data to the lists
    X.append(features)
    y_a.append(arousal)
    y_v.append(valence)


In [8]:
# Concatenate the feature and target arrays
X = np.concatenate(X)
y_a = np.concatenate(y_a)
y_v = np.concatenate(y_v)

In [9]:
# Scale the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Reshape the input data for LSTM
X_reshaped = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

In [10]:
# Define the number of folds for cross-validation
num_folds = 5

# Initialize lists to store evaluation results
mse_v = []
mae_v = []
rmse_v = []
r2_v = []
f1_v = []
accuracy_v = []
precision_v = []
recall_v = []

In [11]:
# Perform tenfold cross-validation
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
for train_index, test_index in kf.split(X_reshaped):
    X_train, X_test = X_reshaped[train_index], X_reshaped[test_index]
    y_v_train, y_v_test = y_v[train_index], y_v[test_index]

    # Build the BiLSTM model for Arousal
    model_v = Sequential()
    model_v.add(Bidirectional(CuDNNLSTM(64, return_sequences=True), input_shape=(X_train.shape[1], 1)))
    model_v.add(Flatten())

    # Add a DNN layer after the BiLSTM
    model_v.add(Dense(32, activation='relu'))

    model_v.add(Dense(1))
    model_v.compile(loss='mse', optimizer=Adam(learning_rate=0.001))

    # Train the Arousal model
    model_v.fit(X_train, y_v_train, epochs=25, batch_size=32)

    # Evaluate the model on the test set
    predictions_v = model_v.predict(X_test)

    # Reshape predictions_a to match the shape of y_a_test
    predictions_v = predictions_v.reshape(y_v_test.shape)

    # Calculate additional evaluation metrics
    mse_v.append(mean_squared_error(y_v_test, predictions_v))
    mae_v.append(mean_absolute_error(y_v_test, predictions_v))
    rmse_v.append(np.sqrt(mean_squared_error(y_v_test, predictions_v)))
    r2_v.append(r2_score(y_v_test, predictions_v))

    # Convert regression predictions to binary labels
    threshold = 0
    binary_predictions_v = (predictions_v >= threshold).astype(int)
    y_v_test_binary = (y_v_test >= threshold).astype(int)

    # Calculate F1-score, accuracy, precision, and recall for binary classification
    f1_v.append(f1_score(y_v_test_binary, binary_predictions_v))
    accuracy_v.append(accuracy_score(y_v_test_binary, binary_predictions_v))
    precision_v.append(precision_score(y_v_test_binary, binary_predictions_v))
    recall_v.append(recall_score(y_v_test_binary, binary_predictions_v))


Epoch 1/25
2654/2654 [==============================] - 58s 19ms/step - loss: 0.0445
Epoch 2/25
2654/2654 [==============================] - 47s 18ms/step - loss: 0.0395
Epoch 3/25
2654/2654 [==============================] - 51s 19ms/step - loss: 0.0386
Epoch 4/25
2654/2654 [==============================] - 47s 18ms/step - loss: 0.0380
Epoch 5/25
2654/2654 [==============================] - 51s 19ms/step - loss: 0.0376
Epoch 6/25
2654/2654 [==============================] - 48s 18ms/step - loss: 0.0372
Epoch 7/25
2654/2654 [==============================] - 49s 18ms/step - loss: 0.0369
Epoch 8/25
2654/2654 [==============================] - 50s 19ms/step - loss: 0.0367
Epoch 9/25
2654/2654 [==============================] - 48s 18ms/step - loss: 0.0365
Epoch 10/25
2654/2654 [==============================] - 49s 19ms/step - loss: 0.0363
Epoch 11/25
2654/2654 [==============================] - 48s 18ms/step - loss: 0.0361
Epoch 12/25
2654/2654 [==============================] - 49s 18

In [12]:

# Calculate average performance across all folds
average_mse_v = np.mean(mse_v)
average_mae_v = np.mean(mae_v)
average_rmse_v = np.mean(rmse_v)
average_r2_v = np.mean(r2_v)
average_f1_v = np.mean(f1_v)
average_accuracy_v = np.mean(accuracy_v)
average_precision_v = np.mean(precision_v)
average_recall_v = np.mean(recall_v)

print(f'Average Arousal MSE: {average_mse_v:.4f}')
print(f'Average Arousal MAE: {average_mae_v:.4f}')
print(f'Average Arousal RMSE: {average_rmse_v:.4f}')
print(f'Average Arousal R2 Score: {average_r2_v:.4f}')
print(f'Average Arousal F1 Score: {average_f1_v:.4f}')
print(f'Average Arousal Accuracy: {average_accuracy_v:.4f}')
print(f'Average Arousal Precision: {average_precision_v:.4f}')
print(f'Average Arousal Recall: {average_recall_v:.4f}')

Average Arousal MSE: 0.0334
Average Arousal MAE: 0.1458
Average Arousal RMSE: 0.1828
Average Arousal R2 Score: 0.3957
Average Arousal F1 Score: 0.8227
Average Arousal Accuracy: 0.7559
Average Arousal Precision: 0.7889
Average Arousal Recall: 0.8602
